# Find the topics about Seoul

### Topic Modeling(LDA) with KOT POI data

Which topic is the most relevant with Seoul?
Let's find it from overview text from KOT POI data about Seoul

In [1]:
import pandas as pd
import numpy as np
import gensim
import nltk
import time
import urllib.request


from matplotlib import pyplot as plt
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from pprint import pprint
# from text_cleaning import lemmatize_and_stem, preprocess, get_aggregate_score, replace_periods

from functions import remove_html, remove_newline

In [2]:
# set database connection
from pymongo import MongoClient 
client = MongoClient('localhost', 27017)

In [3]:
# set database
db = client['cp_seoul']

In [4]:
# set collection
coll = db['seoul_poi']

In [5]:
# test db connection (get first record from db)
cursor = coll.find({})
cursor.next()

{'_id': ObjectId('5db26b80c36b1f23ca16b1d9'),
 'addr1': '38, Donggyo-ro 29-gil, Mapo-gu, Seoul',
 'areacode': '1',
 'cat1': 'A05',
 'cat2': 'A0502',
 'cat3': 'A05020900',
 'contentid': '2480979',
 'contenttypeid': '82',
 'createdtime': '20170214145634',
 'firstimage': 'http://tong.visitkorea.or.kr/cms/resource/87/2479687_image2_1.jpg',
 'firstimage2': 'http://tong.visitkorea.or.kr/cms/resource/87/2479687_image3_1.jpg',
 'mapx': '126.9230428662',
 'mapy': '37.5621620943',
 'masterid': '2479328',
 'modifiedtime': '20190125151640',
 'readcount': '3832',
 'sigungucode': '13',
 'title': '17℃ (17도씨)',
 'zipcode': '03984',
 'directions': '[Subway]<br />\nHongik Univ. Station (Seoul Subway Line 2, Gyeongui-Jungang Line, Airport Railroad), Exit 3.<br />\nWalk for approx. 5 min.<br><br>\n\n[Bus]<br />\nKolon Apt. 106-dong Entrance Bus Stop<br />\nKolon Apt. 104-dong Entrance Bus Stop<br />\nDaemyeong Apt. Bus Stop<br />\nMaeul Mapo 05',
 'dongcode': '27',
 'homepage': '<a title="open in new wind

cf)  
    82: "음식(82)Dining",
    76: "관광지(76)TouristAttractions",
    79: "쇼핑(79)Shopping",
    78: "문화시설(78)Cultural Facilities",
    80: "숙박(80)Accommodation",
    85: "축제/공연/행사(85)Festivals/Events/Performances",
    75: "레포츠(75)Leisure/Sports",
    77: "교통(77)Transportation",    

In [6]:
# load data to dataframe (only Tourist attraction and Culutral Facilities)
cursor = coll.find({'contenttypeid': {"$in": ['76', '78', ]}})
df = pd.DataFrame(list(cursor))

In [7]:
df.head().T

,0,1,2,3,4
_id,5db26b80c36b1f23ca16b1da,5db26b80c36b1f23ca16b1de,5db26b80c36b1f23ca16b1e0,5db26b80c36b1f23ca16b1e1,5db26b80c36b1f23ca16b1e4
addr1,"31, Daehak-ro 12-gil, Jongno-gu, Seoul","50, 63-ro, Yeongdeungpo-gu, Seoul","50, 63-ro, Yeongdeungpo-gu, Seoul","20-1, Samil-daero 8-gil, Jung-gu, Seoul","21, Apgujeong-ro 29-gil, Gangnam-gu, Seoul"
addr2,NaN,NaN,NaN,NaN,NaN
areacode,1,1,1,1,1
cat1,A02,A02,A02,A02,A02
cat2,A0206,A0206,A0205,A0203,A0202
cat3,A02060600,A02060500,A02050600,A02030400,A02020300
contentid,1240735,621155,264122,2590011,1295190
contenttypeid,78,78,76,76,76
createdtime,20110324102837,20080908171332,20021228150750,20190211130745,20110601095512


In [8]:
# get overview text of POIs
overviews = list(df['overview'])

In [9]:
overviews[:5]

['1m Classic Art Hall, located in Daehang-ro is a classic art experience center exclusively for children. The center provides children with the opportunity to enjoy classical music in a fun and interesting way. In the ‘1m Experience Classic’ program (the oldest children’s classical program in the nation), teachers not only provide interesting mini-music learns, but also assist children in trying out a variety of instruments.<br><br>\n\nAt the performance halls at the 1m Classic Art Hall, the stage is just 1m away from the surrounding seats, giving young audience members a more intimate musical experience.',
 '63 City in Yeouido stands 264 meters above sea level and is considered one of Seoul’s most well known landmarks. The 60th floor ‘Sky Deck’ of 63 City Building was remodeled into an art museum, offering a place for art exhibitions, magic shows, and various cultural experiences. Visitors will also be able to enjoy the beautiful scenery of Hangang River through the observatory’s larg

In [10]:
# need to clean the markup characters (<Br>, \n, ...)
overviews = list(map(remove_html, overviews))
overviews = list(map(remove_newline, overviews))

In [11]:
overviews[:5]

['1m Classic Art Hall, located in Daehang-ro is a classic art experience center exclusively for children. The center provides children with the opportunity to enjoy classical music in a fun and interesting way. In the ‘1m Experience Classic’ program (the oldest children’s classical program in the nation), teachers not only provide interesting mini-music learns, but also assist children in trying out a variety of instruments.\n\nAt the performance halls at the 1m Classic Art Hall, the stage is just 1m away from the surrounding seats, giving young audience members a more intimate musical experience.',
 '63 City in Yeouido stands 264 meters above sea level and is considered one of Seoul’s most well known landmarks. The 60th floor ‘Sky Deck’ of 63 City Building was remodeled into an art museum, offering a place for art exhibitions, magic shows, and various cultural experiences. Visitors will also be able to enjoy the beautiful scenery of Hangang River through the observatory’s large window

#### Tokenize with Lemmatization and Stemming

In [12]:
from gensim.utils import simple_preprocess

def lemmatize_and_stem(text):
    """Return Lemmatized and stemmed text"""
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos="v"))


In [13]:
# NLTK Stop words
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
#extends stopwords for this dataset
stopwords.extend([
    "korea",
    "korean",
    "dong",
    "visit",
    "include",
    "area",
    "food",
    "serve",
    "dish",
    "restaurant",
    "offer",
    "center",
    "hotel",
    "guest",
    "product"
])


In [14]:
# preprocess function
def preprocess(text):
    """Return lowercase, lemmatize, and stem text"""
    result = []

    for token in simple_preprocess(text):
        if token not in stopwords:
            result.append(lemmatize_and_stem(token))
    return result

In [15]:
# perform lemmatize_and_stem and remove stop words
overviews_token = list(map(preprocess, overviews))

In [16]:
len(overviews_token)

423

In [17]:
# Create dictionary
id2word = gensim.corpora.Dictionary(overviews_token)

In [18]:
# filtering extreme words
id2word.filter_extremes(no_below=5, no_above=0.30)

In [19]:
# make Term Document Frequency (corpus)
corpus = [id2word.doc2bow(overview) for overview in overviews_token]

In [20]:
len(corpus)

423

In [21]:
# build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus, 
                                            num_topics=5,
                                            id2word=id2word,
                                            passes=20,
                                            alpha='auto')


In [22]:
# print topics
pprint(lda_model.print_topics())

[(0,
  '0.043*"park" + 0.027*"theater" + 0.024*"river" + 0.023*"perform" + '
  '0.022*"hangang" + 0.014*"bridg" + 0.012*"church" + 0.011*"art" + '
  '0.009*"music" + 0.009*"baekj"'),
 (1,
  '0.036*"museum" + 0.030*"art" + 0.025*"tradit" + 0.016*"exhibit" + '
  '0.016*"visitor" + 0.012*"templ" + 0.012*"galleri" + 0.009*"hous" + '
  '0.009*"view" + 0.008*"open"'),
 (2,
  '0.023*"mountain" + 0.023*"hall" + 0.022*"park" + 0.017*"exhibit" + '
  '0.013*"museum" + 0.012*"art" + 0.011*"build" + 0.010*"design" + '
  '0.009*"nation" + 0.009*"room"'),
 (3,
  '0.022*"street" + 0.020*"shop" + 0.017*"visitor" + 0.016*"store" + '
  '0.011*"inform" + 0.011*"tourist" + 0.010*"experi" + 0.009*"intern" + '
  '0.009*"market" + 0.009*"restaur"'),
 (4,
  '0.027*"build" + 0.026*"palac" + 0.025*"king" + 0.024*"gate" + 0.013*"art" + '
  '0.013*"hall" + 0.011*"dynasti" + 0.011*"joseon" + 0.011*"year" + '
  '0.010*"tomb"')]


*Overview text of POIs have 5 topics*

* Namsan [3]
* Park and Nature (Hangang(River)) [5]
* Museum (Traditional and Historical) [2]
* Art (Exhibition and Performs) [1]
* Interesting Area (Hongdae, Gangnam, Garosugil) [4]


#### Visualize the topics-keywords

In [23]:
# import Plotting tools (pyLDAvis)
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
# Draw plot with LDA Model
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/teo/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.062949  0.000845       1        1  27.764534
1      0.011035 -0.104892       2        1  24.210878
4     -0.156488 -0.022133       3        1  23.296261
0      0.036765  0.177281       4        1  13.705537
3      0.171637 -0.051100       5        1  11.022791, topic_info=    Category        Freq       Term       Total  loglift  logprob
162  Default  274.000000       park  274.000000  30.0000  30.0000
573  Default  140.000000      palac  140.000000  29.0000  29.0000
234  Default  124.000000       king  124.000000  28.0000  28.0000
85   Default  120.000000       gate  120.000000  27.0000  27.0000
44   Default  272.000000     museum  272.000000  26.0000  26.0000
51   Default   70.000000      river   70.000000  25.0000  25.0000
459  Default   99.000000    theater   99.000000  24.0000  24.0000
38   Default   64.000000    hangang   64.000000  23.0000  23.0000
66   Default  158.000000   mountain  158.000000  22.0000  22.0000
384  Default   68.000000     street   68.000000  21.0000  21.0000
381  Default   72.000000       shop   72.000000  20.0000  20.0000
19   Default  130.000000    perform  130.000000  19.0000  19.0000
98   Default  169.000000     tradit  169.000000  18.0000  18.0000
32   Default  232.000000      build  232.000000  17.0000  17.0000
383  Default   38.000000      store   38.000000  16.0000  16.0000
59   Default  171.000000    visitor  171.000000  15.0000  15.0000
503  Default   50.000000      bridg   50.000000  14.0000  14.0000
557  Default  104.000000      templ  104.000000  13.0000  13.0000
36   Default  194.000000    exhibit  194.000000  12.0000  12.0000
814  Default   37.000000     church   37.000000  11.0000  11.0000
10   Default  226.000000       hall  226.000000  10.0000  10.0000
688  Default   40.000000     inform   40.000000   9.0000   9.0000
0    Default  308.000000        art  308.000000   8.0000   8.0000
558  Default   51.000000       tomb   51.000000   7.0000   7.0000
297  Default   43.000000    tourist   43.000000   6.0000   6.0000
223  Default   28.000000      baekj   28.000000   5.0000   5.0000
23   Default   44.000000       seat   44.000000   4.0000   4.0000
474  Default   40.000000     villag   40.000000   3.0000   3.0000
525  Default   28.000000      sport   28.000000   2.0000   2.0000
253  Default   53.000000       educ   53.000000   1.0000   1.0000
..       ...         ...        ...         ...      ...      ...
407   Topic5   19.561567       zone   27.039970   1.8815  -4.7535
836   Topic5    9.985838  entertain   13.995687   1.8676  -5.4259
657   Topic5    4.660948        toy    6.537839   1.8668  -6.1879
631   Topic5    5.919949       town    8.331577   1.8635  -5.9488
680   Topic5   13.463044    gangnam   19.684723   1.8253  -5.1272
687   Topic5    6.773690    english   10.243473   1.7916  -5.8141
146   Topic5    5.470443      price    8.323439   1.7855  -6.0278
699   Topic5    7.303346     subway   11.125589   1.7843  -5.7388
698   Topic5   13.161051    station   20.351109   1.7693  -5.1499
381   Topic5   44.671139       shop   72.632385   1.7191  -3.9278
688   Topic5   24.922173     inform   40.854317   1.7110  -4.5114
297   Topic5   24.137491    tourist   43.460701   1.6171  -4.5433
855   Topic5    8.325959       sell   13.126004   1.7500  -5.6077
529   Topic5   10.927651   district   19.653683   1.6182  -5.3358
118   Topic5   21.158897     intern   55.205193   1.2462  -4.6751
348   Topic5   18.667641     around   50.217052   1.2156  -4.8003
59    Topic5   37.642612    visitor  171.737671   0.6874  -4.0990
474   Topic5   16.040735     villag   40.441963   1.2805  -4.9520
282   Topic5   14.289809      famou   34.764637   1.3162  -5.0676
528   Topic5   19.439774      world   69.070572   0.9374  -4.7598
7     Topic5   21.710993     experi  108.757607   0.5939  -4.6493
292   Topic5   11.884269      plant   30.083271   1.2765  -5.2519
127   Topic5   

*5 topics*

* 1 - Art (Exhibition and Performs)
* 2 - Museum (Traditional and Historical)
* 3 - Namsan
* 4 - Interesting Area (Hongdae, Gangnam, Garosugil)
* 5 - Park and Nature (Hangang(River))






# Conclusion

## LDA modeling can find meaningful topics from overview text